<a href="https://colab.research.google.com/github/rahiakela/opencv-projects-and-guide/blob/main/mastering-opencv4-with-python/11-face-detection-tracking-and-recognition/4_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Face recognition

With the development of computer vision, machine learning, and deep learning, face recognition has become a hot topic. **Face recognition can be applied to a wide range of uses, including crime prevention, surveillance, forensic applications, biometrics, and, more recently, in social networks.** Automatic face recognition has various challenges, such as occlusions, appearance variations, expression, aging, and scale variations. Following its success with object recognition, CNNs have been widely used for face recognition.

In this notebook, we will see the functionality that OpenCV offers in connection with face recognition, and will also explore some deep learning approaches, which can be easily integrated into your computer vision projects to perform state-of-the-art face recognition results.

## Setup

In [1]:
%%shell

pip install dlib

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

import dlib

In [ ]:
%%shell

wget https://github.com/PacktPublishing/Mastering-OpenCV-4-with-Python/raw/master/Chapter11/01-chapter-content/face_recognition/dlib_face_recognition_resnet_model_v1.dat
wget https://github.com/PacktPublishing/Mastering-OpenCV-4-with-Python/raw/master/Chapter11/01-chapter-content/face_recognition/shape_predictor_5_face_landmarks.dat

wget https://github.com/PacktPublishing/Mastering-OpenCV-4-with-Python/raw/master/Chapter11/01-chapter-content/face_recognition/jared_1.jpg
wget https://github.com/PacktPublishing/Mastering-OpenCV-4-with-Python/raw/master/Chapter11/01-chapter-content/face_recognition/jared_2.jpg
wget https://github.com/PacktPublishing/Mastering-OpenCV-4-with-Python/raw/master/Chapter11/01-chapter-content/face_recognition/jared_3.jpg
wget https://github.com/PacktPublishing/Mastering-OpenCV-4-with-Python/raw/master/Chapter11/01-chapter-content/face_recognition/jared_4.jpg
wget https://github.com/PacktPublishing/Mastering-OpenCV-4-with-Python/raw/master/Chapter11/01-chapter-content/face_recognition/obama.jpg

In [13]:
def display_images(images, columns=5, width=20, height=8, max_images=15, label_wrap_length=50, label_font_size=8):

    if not images:
        print("No images to display.")
        return 

    if len(images) > max_images:
        print(f"Showing {max_images} images of {len(images)}:")
        images=images[0:max_images]

    height = max(height, int(len(images) / columns) * height)
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):

        plt.subplot(len(images) / columns + 1, columns, i + 1)
        plt.imshow(image)

        if hasattr(image, 'filename'):
            title=image.filename
            if title.endswith("/"): title = title[0:-1]
            title=os.path.basename(title)
            title=textwrap.wrap(title, label_wrap_length)
            title="\n".join(title)
            plt.title(title, fontsize=label_font_size); 

def display_images2(images, names, columns=5, width=20, height=8, max_images=15, label_wrap_length=50, label_font_size=8):

    if not images:
        print("No images to display.")
        return 

    if len(images) > max_images:
        print(f"Showing {max_images} images of {len(images)}:")
        images=images[0:max_images]

    height = max(height, int(len(images) / columns) * height)
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):

        plt.subplot(len(images) / columns + 1, columns, i + 1)
        plt.imshow(image)
        plt.title(names[i], fontsize=label_font_size); 

        if hasattr(image, 'filename'):
            title=image.filename
            if title.endswith("/"): title = title[0:-1]
            title=os.path.basename(title)
            title=textwrap.wrap(title, label_wrap_length)
            title="\n".join(title)
            plt.title(title, fontsize=label_font_size); 

## Face recognition with dlib

**Dlib offers a high-quality face recognition algorithm based on deep learning. Dlib implements a face recognition algorithm that offers state-of-the-art accuracy.** More specifically, the model has an accuracy of 99.38% on the labeled faces in the wild database.

The implementation of this algorithm is based on the ResNet-34 network proposed in the paper Deep Residual Learning for Image Recognition (2016), which was trained using three million faces.

**This network is trained in a way that generates a 128-dimensional (128D) descriptor, used to quantify the face. The training step is performed using triplets.**

**A single triplet training example is composed of three images. Two of them correspond to the same person. The network generates the 128D descriptor for each of the images, slightly modifying the neural network weights in order to make the two vectors that correspond to the same person closer and the feature vector from the other person further away.** The triplet loss function
formalizes this and tries to push the 128D descriptor of two images of the same person closer together, while pulling the 128D descriptor of two images of different people further apart.

This process is repeated millions of times for millions of images of thousands of different people and finally, it is able to generate a 128D descriptor for each person. So, the final 128D descriptor is good encoding for the following reasons:

- The generated 128D descriptors of two images of the same person are quite
similar to each other.

- The generated 128D descriptors of two images of different people are very
different.

**Therefore, making use of the dlib functionality, we can use a pre-trained model to map a face into a 128D descriptor. Afterward, we can use these feature vectors to perform face recognition.**

Let's see how to calculate the 128D descriptor, used to
quantify the face. The process is quite simple.

Now let's calculate the encodings for every face of the image.As you can guess, it calculate the 128D descriptor for each face in the image:

In [4]:
# Load shape predictor, face enconder and face detector using dlib library
pose_predictor_5_point = dlib.shape_predictor("shape_predictor_5_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
detector = dlib.get_frontal_face_detector()

In [5]:
def face_encodings(face_image, number_of_times_to_upsample=1, num_jitters=1):
  """Returns the 128D descriptor for each face in the image"""

  # Detect faces
  face_locations = detector(face_image, number_of_times_to_upsample)
  # Detected landmarks
  raw_landmarks = [pose_predictor_5_point(face_image, face_location) for face_location in face_locations]

  # Calculate the face encoding for every detected face using the detected landmarks for each one
  return [np.array(face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters)) for raw_landmark_set in raw_landmarks]

As you can see, the key point is to calculate the face encoding for every detected face using the detected landmarks for each one, calling dlib
the `face_encoder.compute_face_descriptor()` function.

The `num_jitters parameter` sets the number of times each face will be randomly jittered, and the average 128D descriptor calculated each time will be returned.

Once the faces are encoded, the next step is to perform the recognition.

The recognition can be easily computed using some kind of distance metrics computed using the `128D` descriptors. Indeed, if two face descriptor vectors have a Euclidean distance between them that is less than `0.6`, they can be considered to belong to the same person. Otherwise, they are from different people.

The Euclidean distance can be calculated using numpy.linalg.norm().

We compare four images against another image. To compare the faces, we have coded two functions: 

- `compare_faces()` and
- `compare_faces_ordered()`. 

The compare_faces() function returns the distance when
comparing a list of face encodings against a candidate to check:

In [6]:
def compare_faces(encodings, encoding_to_check):
  """Returns the distances when comparing a list of face encodings against a candidate to check"""

  return list(np.linalg.norm(encodings - encoding_to_check, axis=1))

The compare_faces_ordered() function returns the ordered distances and the
corresponding names when comparing a list of face encodings against a candidate to check:

In [7]:
def compare_faces_ordered(encodings, face_names, encoding_to_check):
  """Returns the ordered distances and names when comparing a list of face encodings against a candidate to check"""

  distances = list(np.linalg.norm(encodings - encoding_to_check, axis=1))

  return zip(*sorted(zip(distances, face_names)))

Therefore, the first step in comparing four images against another image is to load all of them and convert to RGB (dlib format):

In [8]:
# Load image
known_image_1 = cv2.imread("jared_1.jpg")
known_image_2 = cv2.imread("jared_2.jpg")
known_image_3 = cv2.imread("jared_3.jpg")
known_image_4 = cv2.imread("obama.jpg")
unknown_image = cv2.imread("jared_4.jpg")

# Convert image from BGR (OpenCV format) to RGB (dlib format)
known_image_1 = known_image_1[:, :, ::-1]
known_image_2 = known_image_2[:, :, ::-1]
known_image_3 = known_image_3[:, :, ::-1]
known_image_4 = known_image_4[:, :, ::-1]
unknown_image = unknown_image[:, :, ::-1]

# Create names for each loaded image
names = ["Jared 1", "Jared 2", "Jared 3", "Obama"]

The next step is to compute the encodings for each image:

In [9]:
# Create the encodings
known_image_1_encoding = face_encodings(known_image_1)[0]
known_image_2_encoding = face_encodings(known_image_2)[0]
known_image_3_encoding = face_encodings(known_image_3)[0]
known_image_4_encoding = face_encodings(known_image_4)[0]

known_encodings = [known_image_1_encoding, known_image_2_encoding, known_image_3_encoding, known_image_4_encoding]

unknown_encoding = face_encodings(unknown_image)[0]

And finally, you can compare the faces using the previous functions. For example, let's make use of the `compare_faces_ordered()` function:

In [10]:
computed_distances_ordered, ordered_names = compare_faces_ordered(known_encodings, names, unknown_encoding)
print(computed_distances_ordered)
print(ordered_names)

(0.3913188139970749, 0.3998326700265651, 0.41041538984538495, 0.9053701470825026)
('Jared 3', 'Jared 1', 'Jared 2', 'Obama')


The first three values are less than 0.6. This means that the first three imagescan be considered from the same person.

The fourth value obtained means that the fourth image is not the same
person.

In [11]:
# Compare faces
computed_distances = compare_faces(known_encodings, unknown_encoding)
print(computed_distances)

[0.3998326700265651, 0.41041538984538495, 0.3913188139970749, 0.9053701470825026]


<img src='https://github.com/rahiakela/img-repo/blob/master/object-detection-images/face-recognition.png?raw=1' width='800'/>

We can see that the first three images can be considered from the same person (the obtained values are less than 0.6), while the fourth image can be
considered from another person (the obtained value is greater than 0.6).

## Face recognition with face_recognition